In [59]:
import numpy as np
from numpy import array
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import os
import gc
import json
import pickle
import glob

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import SGDClassifier, SGDRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")
from subprocess import check_output
%matplotlib inline

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE

from os import path
from PIL import Image

import time
import warnings
import sqlite3
from sqlalchemy import create_engine # database connection
import csv
import os
warnings.filterwarnings("ignore")
import datetime as dt
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
# from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.cross_validation import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier



from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
# from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve


# DLL
from tensorflow import keras
import tensorflow as tf
import tensorflow.compat.v1 as tf1

from keras.layers.embeddings import Embedding
from keras.layers.merge import add

from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils
from keras.layers.wrappers import Bidirectional

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Attention, 
                                     Layer,
                                     Add, concatenate, 
                                     Input, 
                                     Dense,  
                                     LSTM, Bidirectional, GRU,
                                     ZeroPadding2D, 
                                     Convolution2D, Conv2D, 
                                     GlobalAveragePooling2D, GlobalAvgPool2D, GlobalMaxPooling2D, GlobalMaxPool2D, 
                                     AveragePooling2D, AvgPool2D, MaxPooling2D, MaxPool2D,
                                     Flatten,
                                     BatchNormalization, 
                                     Dropout)

from tensorflow.keras.layers import (Activation, 
                                     ReLU, 
                                     LeakyReLU, 
                                     Softmax)

from tensorflow.keras.optimizers import (SGD,
                                         Adam,
                                         Adagrad,
                                         Adadelta,
                                         RMSprop,
                                         Nadam)

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

from tqdm import tqdm

In [ ]:
import tensorflow_addons as tfa

In [60]:
def load_txt(name):
    with open(name, "r") as f:
        return f.read()

def load_pickle(name):
    with open(name, "rb") as handle:
        return pickle.load(handle)
    
def dump_pickle(name, file):
    with open(name, "wb") as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
drug_features_data = pd.read_csv("../ix_word/final_features.csv", index_col="Unnamed: 0")
drug_features_data.head(2)

In [ ]:
drug_features_data.info()

In [ ]:
drug_features_data_X = drug_features_data.drop(columns=["base_score"])
drug_features_data_y = drug_features_data["base_score"]
drug_features_data_X.shape, drug_features_data_y.shape

In [ ]:
train_data_X = drug_features_data_X.iloc[:10000, :].drop(columns=["drug_approved_by_UIC"]).values
print(train_data_X.shape)
train_data_y = drug_features_data_y.iloc[:10000].values
print(train_data_y.shape)
validate_data_X = drug_features_data_X.iloc[10000:, :].drop(columns=["drug_approved_by_UIC"]).values
print(validate_data_X.shape)
validate_data_y = drug_features_data_y.iloc[10000:].values
print(validate_data_y.shape)

In [ ]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(train_data_X)
y = scaler_y.fit_transform(train_data_y.reshape((10000, 1)))

In [ ]:
drug_features_data_X.columns

In [ ]:
col = list(drug_features_data_X.columns)
col.pop(col.index("drug_approved_by_UIC"))
X = pd.DataFrame(X, columns = col)
lstm_X = X.loc[:, "review_0":"review_767"]
lstm_X.head(2)

In [36]:
drug_train_data = pd.read_csv("drug_preprocess.csv", index_col="Unnamed: 0")
drug_train_data.head(2)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_len,review_n_words,review_by_patient_filter,review_by_patient_unique_word,len_review_by_patient_unique_word
10091,223808,Depakote ER,Epilepsy,"""After trying 4 different meds this one was th...",8,1-Apr-08,15,7.633384,370,74,after trying 4 different meds this one was the...,"['effect', 'trying', '4', 'possible', 'quit', ...",35
1009,35602,Anexsia,Pain,"""Worked better than Tylenol w/codeine #3 and C...",10,1-Apr-08,19,6.127475,88,14,worked better than tylenol w codeine 3 and ca...,"['3', 'carisoprodol', 'better', 'codeine', 'ty...",10


In [39]:
train_data = drug_train_data.iloc[:10000, :]
test_data = drug_train_data.iloc[10000:, :]

In [40]:
train_data.shape, test_data.shape

((10000, 13), (22165, 13))

In [ ]:
all_train_name_use_review = list(drug_train_data["combine_name_use_review"])

In [ ]:
def ixtoword_wordtoix():
    ixtoword = {}
    wordtoix = {}
    ix = 1
    for w in vocab:
        wordtoix[w] = ix
        ixtoword[ix] = w
        ix += 1
    return ixtoword, wordtoix
if 'wordtoix.pkl' in os.listdir("../ix_word"):
    print("load")
    ixtoword = load_pickle('../ix_word/ixtoword.pkl')
    wordtoix = load_pickle('../ix_word/wordtoix.pkl')
else:
    ixtoword, wordtoix = ixtoword_wordtoix()
    dump_pickle('../ix_word/ixtoword.pkl', ixtoword)
    dump_pickle('../ix_word/wordtoix.pkl', wordtoix)
    print("dump")
vocab_size = len(ixtoword) + 1
print('size of vocabulary = %d in our trainig data' % (vocab_size))

In [ ]:
mlp_X= X.loc[:, "effectiveness_rating":"len_review_by_patient_unique_word"]
mlp_X.head(2)

In [ ]:
# embedding_dim =768
# vocab_size = 2986
# max_length = 83
# input_shape = 2048
# # CNN
# inputs1 = Input(shape=(input_shape,))

# fe1 = Dropout(0.2)(inputs1)
# fe1 = Dense(256)(fe1)
# fe1 = BatchNormalization()(fe1)
# fe1 = ReLU()(fe1)


# # RNN - input --> bert embedding --> LSTM --> LSTM
# inputs2 = Input(shape=(max_length,))
# se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
# se1 = Dropout(0.2)(se1)
# se1 = BatchNormalization()(se1)
# se1 = Bidirectional(LSTM(128))(se1)


# # Add both LSTM and CNN
# decoder1 = concatenate([fe1, se1])
# decoder1 = Dropout(0.2)(decoder1)
# decoder1 = Dense(2048)(decoder1)
# decoder1 = BatchNormalization()(decoder1)
# decoder1 = ReLU()(decoder1)
# decoder1 = Dropout(0.2)(decoder1)
# outputs = Dense(vocab_size, activation="softmax")(decoder1)


# m1 = Model(inputs=[inputs1, inputs2], outputs=outputs)
# m1.summary(), m1.layers

In [ ]:
embedding_dim =768
vocab_size = 2986
max_length = 83
lstm_input_shape = 768
mlp_input_shape = 3

# RNN - input --> bert embedding --> LSTM --> LSTM
inputs1 = Input(shape=(1, lstm_input_shape,)) # 768
# em1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs1)
se1 = Dropout(0.2)(inputs1)
se1 = BatchNormalization()(se1)
se1 = Bidirectional(LSTM(128))(se1)
se2 = Dropout(0.2)(inputs1)
se2 = BatchNormalization()(se2)
se2 = Bidirectional(LSTM(64))(se2)

inputs2 = Input(shape=(mlp_input_shape,)) # 3

fe1 = Dropout(0.2)(inputs2)
fe1 = Dense(32)(fe1)
fe1 = BatchNormalization()(fe1)
fe1 = ReLU()(fe1)
fe1 = Dropout(0.2)(fe1)
fe1 = Dense(128)(fe1)
fe1 = BatchNormalization()(fe1)
fe1 = ReLU()(fe1)


# Add both LSTM and CNN
decoder1 = concatenate([se1, se2, fe1])
decoder1 = Dropout(0.2)(decoder1)
decoder1 = BatchNormalization()(decoder1)
decoder1 = Dense(128)(decoder1)
decoder1 = ReLU()(decoder1)
decoder1 = Dropout(0.2)(decoder1)
decoder1 = BatchNormalization()(decoder1)
decoder1 = Dense(32)(decoder1)
decoder1 = ReLU()(decoder1)
decoder1 = Dropout(0.2)(decoder1)
decoder1 = BatchNormalization()(decoder1)
outputs = Dense(1, kernel_initializer='normal', activation='linear')(decoder1)

m1 = Model(inputs=[inputs1, inputs2], outputs=outputs)
m1.summary(), m1.layers

In [ ]:
# #                                                  34,         30
# def data_generator(description, photos, wordtoix, max_length, num_photos_per_batch):
#     X1, X2, y = list(), list(), list()
# #     X1, X2, X3, y = list(), list(), list(), list()
#     n=0
#     # loop for ever over images
#     while 1:
#         for key, desc_list in description.items():
#             n+=1
#             # retrieve the photo feature
#             photo = photos[key]
# #             photo2 = simple_model_feature[key+'.jpg']
# #             photo2 = encode2(f'data/images/{key}.jpg')
# #             photo2 = img_to_array(load_img(f'data/images/{key}.jpg', target_size=(299, 299)))

#             for desc in desc_list:
#                 # print(desc)
#                 # encode the sequence
# #                 seq = 
#                 seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
#                 # split one sequence into multiple X, y pairs
#                 for i in range(1, len(seq)):
#                     # split into input and output pair
#                     in_seq, out_seq = seq[:i], seq[i]
#                     # pad input sequence
#                     in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
#                     # encode output sequence
#                     out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
#                     # store
#                     X1.append(photo) # All photos
#                     X2.append(in_seq) # 
# #                     X3.append(in_seq)
#                     y.append(out_seq)

#             if n==num_photos_per_batch:
# #                 print(n)
#                 yield ([array(X1), array(X2)], array(y))
# #                 print("hello")
#                 X1, X2, y = list(), list(), list()
#                 n=0

In [ ]:
#                    768,    3,   1
def data_generator(lstm_X, mlp_X, out, num_record_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    while 1:
        for i in range(lstm_X.shape[0]):
            n+=1
            # retrieve the photo feature
            a = lstm_X[i]
            X1.append(a)
            b = mlp_X[i]
            X2.append(b)
            y.append(out[i])

            if n==num_record_per_batch:
                print(n)
                print(array(X1).shape)
                print(array(X2).shape)
                print("ggg", len(y))
                yield ([array(X1), array(X2)], array(y))
                print("hello")
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
METRICS = [
#     tf.keras.metrics.BinaryAccuracy(name="accuracy"),
    tf.keras.metrics.Precision(name = "precision"),
    tf.keras.metrics.Recall(name="recall"),
#     tf.keras.metrics.mean_absolute_error("mean_absolute_error"),
#     tf.keras.metrics.mean_squared_error(name="mean_squared_error"),
    tfa.metrics.RSquare(name="r_square")
]
# m1.compile(loss='mean_squared_error', optimizer='adam')
m1.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0001), metrics=["accuracy", METRICS])

In [ ]:
m1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
m1.fit([lstm_X.values, mlp_X.values], y, epochs=500, batch_size=32, validation_split = 0.2)

In [ ]:
# m1.fit([lstm_X.values, mlp_X.values], y, epochs=100, batch_size=2)

In [ ]:
#tf.config.experimental_run_functions_eagerly(True)
epochs = 200
batch_size = 2
steps = X.shape[0]//batch_size

for i in range(1, epochs+1):
    print(f"epoch no {i}")
    generator = data_generator(lstm_X.values, mlp_X.values, y, batch_size)
    m1.fit(generator, epochs=1, batch_size=batch_size, steps_per_epoch=steps, verbose=1) 
    m1.save(f"model/arm_{i}.h5")

In [ ]:
# #tf.config.experimental_run_functions_eagerly(True)
# epochs = 200
# batch_size = 2
# steps = len(train_img_capt)//batch_size

# for i in range(1, epochs+1):
#     print(f"epoch no {i}")
#     generator = data_generator(train_img_capt, train_features, wordtoix, max_length, batch_size)
#     m1.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1) 
#     m1.save(f"model/tipa_{i}.h5")

In [61]:
def ixtoword_wordtoix():
    ixtoword = {}
    wordtoix = {}
    ix = 1
    for w in vocab:
        wordtoix[w] = ix
        ixtoword[ix] = w
        ix += 1
    return ixtoword, wordtoix
if 'wordtoix.pkl' in os.listdir("../ix_word"):
    print("load")
    ixtoword = load_pickle('../ix_word/ixtoword.pkl')
    wordtoix = load_pickle('../ix_word/wordtoix.pkl')
else:
    ixtoword, wordtoix = ixtoword_wordtoix()
    dump_pickle('../ix_word/ixtoword.pkl', ixtoword)
    dump_pickle('../ix_word/wordtoix.pkl', wordtoix)
    print("dump")
vocab_size = len(ixtoword) + 1
print('size of vocabulary = %d in our trainig data' % (vocab_size))

load
size of vocabulary = 3167 in our trainig data


In [77]:
m1 = keras.models.load_model("model/main_1.h5")

In [78]:
m1.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 650)]        0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 5)]          0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 650, 768)     2432256     ['input_9[0][0]']                
                                                                                                  
 dropout_38 (Dropout)           (None, 5)            0           ['input_10[0][0]']               
                                                                                            

In [6]:
m1.inputs

[<KerasTensor: shape=(None, 650) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'input_2')>]

In [45]:
train_data = drug_train_data.iloc[:30000, :]
validate_data = drug_train_data.iloc[30000:, :]

In [56]:
validate_data[:100].head(1).iloc[0, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]

review_by_patient_filter             i  039 m 17  been on the patch for over a year...
effectiveness_rating                                                                 9
number_of_times_prescribed                                                           7
review_len                                                                         792
review_n_words                                                                     135
len_review_by_patient_unique_word                                                   66
base_score                                                                    6.607445
Name: 28324, dtype: object

In [76]:
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m1.predict([lst1, lst2], verbose=0, use_multiprocessing=True)
        print(seq.shape, numerical_data.shape, y, ans[0,0], "\n\n")
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[:100])

(1, 650) (1, 5) 6.607445106972985 0.965745 


(1, 650) (1, 5) 6.607445106972985 0.965745 


(1, 650) (1, 5) 6.998161370910088 0.965745 


(1, 650) (1, 5) 7.511502103994039 0.965745 


(1, 650) (1, 5) 7.129246811243481 0.965745 


(1, 650) (1, 5) 8.7151200080925 0.965745 


(1, 650) (1, 5) 8.7151200080925 0.965745 


(1, 650) (1, 5) 6.3732589822241685 0.965745 


(1, 650) (1, 5) 5.648499094454326 0.965745 


(1, 650) (1, 5) 6.207302708748353 0.965745 


(1, 650) (1, 5) 5.025271726702564 0.965745 


(1, 650) (1, 5) 7.106682637574135 0.965745 


(1, 650) (1, 5) 7.170997223562014 0.965745 


(1, 650) (1, 5) 8.215482906027418 0.965745 


(1, 650) (1, 5) 6.71052568386338 0.965745 


(1, 650) (1, 5) 5.799266276454582 0.965745 


(1, 650) (1, 5) 6.607445106972985 0.965745 


(1, 650) (1, 5) 8.59389418591238 0.965745 


(1, 650) (1, 5) 8.748627336347578 0.965745 


(1, 650) (1, 5) 6.562115937095545 0.965745 


(1, 650) (1, 5) 6.382068816107202 0.965745 


(1, 650) (1, 5) 8.709532579215772 0.965

In [84]:
m1 = keras.models.load_model("model/main_1.h5")
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m1.predict([seq, numerical_data], verbose=0, use_multiprocessing=True)
        print(seq.shape, numerical_data.shape, y, ans[0,0], "\n\n")
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[:100])

(1, 650) (1, 5) 6.607445106972985 1.6042988 


(1, 650) (1, 5) 6.607445106972985 1.6042988 


(1, 650) (1, 5) 6.998161370910088 1.6908308 


(1, 650) (1, 5) 7.511502103994039 1.4805044 


(1, 650) (1, 5) 7.129246811243481 0.85865265 


(1, 650) (1, 5) 8.7151200080925 1.6018822 


(1, 650) (1, 5) 8.7151200080925 1.9212229 


(1, 650) (1, 5) 6.3732589822241685 1.4713087 


(1, 650) (1, 5) 5.648499094454326 2.2049105 


(1, 650) (1, 5) 6.207302708748353 1.6835444 


(1, 650) (1, 5) 5.025271726702564 2.0753531 


(1, 650) (1, 5) 7.106682637574135 3.2123244 


(1, 650) (1, 5) 7.170997223562014 1.8713613 


(1, 650) (1, 5) 8.215482906027418 1.1941838 


(1, 650) (1, 5) 6.71052568386338 -4.680144 


(1, 650) (1, 5) 5.799266276454582 1.3061091 


(1, 650) (1, 5) 6.607445106972985 2.7381396 


(1, 650) (1, 5) 8.59389418591238 1.6137276 


(1, 650) (1, 5) 8.748627336347578 1.6510055 


(1, 650) (1, 5) 6.562115937095545 1.3411677 


(1, 650) (1, 5) 6.382068816107202 2.0041707 


(1, 650) (1, 5) 8

In [85]:
m2 = keras.models.load_model("model/main_2.h5")
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m2.predict([seq, numerical_data], verbose=0, use_multiprocessing=True)
        print(seq.shape, numerical_data.shape, y, ans[0,0], "\n\n")
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[:100])

(1, 650) (1, 5) 6.607445106972985 5.6228223 


(1, 650) (1, 5) 6.607445106972985 5.6228223 


(1, 650) (1, 5) 6.998161370910088 4.729066 


(1, 650) (1, 5) 7.511502103994039 5.2125044 


(1, 650) (1, 5) 7.129246811243481 3.2089 


(1, 650) (1, 5) 8.7151200080925 4.0209537 


(1, 650) (1, 5) 8.7151200080925 4.197741 


(1, 650) (1, 5) 6.3732589822241685 5.2359653 


(1, 650) (1, 5) 5.648499094454326 4.250122 


(1, 650) (1, 5) 6.207302708748353 4.7970595 


(1, 650) (1, 5) 5.025271726702564 4.7350416 


(1, 650) (1, 5) 7.106682637574135 7.5222597 


(1, 650) (1, 5) 7.170997223562014 4.8159113 


(1, 650) (1, 5) 8.215482906027418 4.8457966 


(1, 650) (1, 5) 6.71052568386338 6.6883106 


(1, 650) (1, 5) 5.799266276454582 3.98814 


(1, 650) (1, 5) 6.607445106972985 5.6954494 


(1, 650) (1, 5) 8.59389418591238 3.8138866 


(1, 650) (1, 5) 8.748627336347578 1.2907951 


(1, 650) (1, 5) 6.562115937095545 7.6076784 


(1, 650) (1, 5) 6.382068816107202 4.7799854 


(1, 650) (1, 5) 8.70953257

In [87]:
m3 = keras.models.load_model("model/main_3.h5")
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m3.predict([seq, numerical_data], verbose=0, use_multiprocessing=True)
        print(seq.shape, numerical_data.shape, y, ans[0,0], "\n\n")
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[100:200])

(1, 650) (1, 5) 7.431629988810775 2.131108 


(1, 650) (1, 5) 6.198885281714226 2.76837 


(1, 650) (1, 5) 6.168992330132397 0.11713466 


(1, 650) (1, 5) 5.1337406888149335 3.405741 


(1, 650) (1, 5) 5.112275555480224 4.169373 


(1, 650) (1, 5) 6.935684111767527 5.8402452 


(1, 650) (1, 5) 7.053457112018454 1.3210908 


(1, 650) (1, 5) 5.283595657658187 7.9139714 


(1, 650) (1, 5) 1.9856152995334035 -5.7613916 


(1, 650) (1, 5) 7.390379335933776 5.6411343 


(1, 650) (1, 5) 6.930195283510674 0.6300534 


(1, 650) (1, 5) 8.332361850351921 5.445722 


(1, 650) (1, 5) 8.753452412523746 1.2738402 


(1, 650) (1, 5) 6.127475169414972 1.5140882 


(1, 650) (1, 5) 6.1731344641759245 1.4716519 


(1, 650) (1, 5) 4.839710405165771 2.1365492 


(1, 650) (1, 5) 7.323018196415763 4.098969 


(1, 650) (1, 5) 6.920059185010526 1.416188 


(1, 650) (1, 5) 6.633970466631683 6.3925734 


(1, 650) (1, 5) 7.828314657173991 1.5604167 


(1, 650) (1, 5) 6.577285637571317 4.3685513 


(1, 650) (1, 5) 

In [95]:
m4 = keras.models.load_model("model/main_4.h5")
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        if type(text) is float:
            text = ""
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m4.predict([seq, numerical_data], verbose=0, use_multiprocessing=True)
        print(seq.shape, numerical_data.shape, y, ans[0,0], "\n\n")
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[:500])

(1, 650) (1, 5) 6.607445106972985 2.604419 


(1, 650) (1, 5) 6.607445106972985 2.604419 


(1, 650) (1, 5) 6.998161370910088 3.2728148 


(1, 650) (1, 5) 7.511502103994039 3.2919075 


(1, 650) (1, 5) 7.129246811243481 7.125782 


(1, 650) (1, 5) 8.7151200080925 4.867945 


(1, 650) (1, 5) 8.7151200080925 3.5363402 


(1, 650) (1, 5) 6.3732589822241685 3.1120136 


(1, 650) (1, 5) 5.648499094454326 5.7149487 


(1, 650) (1, 5) 6.207302708748353 4.4201536 


(1, 650) (1, 5) 5.025271726702564 4.947669 


(1, 650) (1, 5) 7.106682637574135 1.9414752 


(1, 650) (1, 5) 7.170997223562014 4.3528247 


(1, 650) (1, 5) 8.215482906027418 9.068916 


(1, 650) (1, 5) 6.71052568386338 7.6055655 


(1, 650) (1, 5) 5.799266276454582 4.552628 


(1, 650) (1, 5) 6.607445106972985 6.073137 


(1, 650) (1, 5) 8.59389418591238 -1.4125428 


(1, 650) (1, 5) 8.748627336347578 3.5252745 


(1, 650) (1, 5) 6.562115937095545 1.5327892 


(1, 650) (1, 5) 6.382068816107202 2.9374673 


(1, 650) (1, 5) 8.7095325

(1, 650) (1, 5) 8.811472745889803 3.3670356 


(1, 650) (1, 5) 8.022968937262847 7.1439295 


(1, 650) (1, 5) 6.562115937095545 5.948612 


(1, 650) (1, 5) 8.735985510612334 9.468255 


(1, 650) (1, 5) 6.190498156775787 5.430221 


(1, 650) (1, 5) 7.602357260334827 7.873962 


(1, 650) (1, 5) 8.74138683827009 6.7206774 


(1, 650) (1, 5) 7.493869596318778 3.7070856 


(1, 650) (1, 5) 6.759892234844327 5.2719793 


(1, 650) (1, 5) 8.29845086437032 3.3075469 


(1, 650) (1, 5) 5.95227783477598 -0.8494224 


(1, 650) (1, 5) 8.482883456649951 5.0175395 


(1, 650) (1, 5) 6.3732589822241685 1.7693703 


(1, 650) (1, 5) 7.648306726574672 1.4143138 


(1, 650) (1, 5) 7.493869596318778 3.7070856 


(1, 650) (1, 5) 5.95227783477598 -0.8494224 


(1, 650) (1, 5) 3.4718572059490853 3.6604102 


(1, 650) (1, 5) 6.607445106972985 2.8680246 


(1, 650) (1, 5) 7.921906947449914 3.4489257 


(1, 650) (1, 5) 6.3995078738655895 -0.07052642 


(1, 650) (1, 5) 7.92459471510565 3.1183631 


(1, 650) (1, 5)

(1, 650) (1, 5) 5.991446166933935 2.7850375 


(1, 650) (1, 5) 6.245772451546388 9.52013 


(1, 650) (1, 5) 5.717921805828719 3.5946596 


(1, 650) (1, 5) 3.433064691451144 4.4817944 


(1, 650) (1, 5) 6.181254371017418 3.360627 


(1, 650) (1, 5) 5.1385587970376365 7.234438 


(1, 650) (1, 5) 6.765825685915322 3.9160268 


(1, 650) (1, 5) 3.916649591116808 4.5488195 


(1, 650) (1, 5) 4.878171957939991 2.805763 


(1, 650) (1, 5) 5.433997586477339 -1.5065305 


(1, 650) (1, 5) 5.433997586477339 -1.5065305 


(1, 650) (1, 5) 6.765825685915322 3.9160268 


(1, 650) (1, 5) 8.323452146415569 8.536673 


(1, 650) (1, 5) 5.691791949528471 5.221564 


(1, 650) (1, 5) 5.47795598012679 8.482783 


(1, 650) (1, 5) 8.743005258264253 6.601832 


(1, 650) (1, 5) 5.675766931338512 6.774074 


(1, 650) (1, 5) 8.743005258264253 6.601832 


(1, 650) (1, 5) 8.154949057424771 7.185757 


(1, 650) (1, 5) 6.124057251494149 4.6361156 


(1, 650) (1, 5) 5.675766931338512 7.318663 


(1, 650) (1, 5) 7.171574

In [96]:
m5 = keras.models.load_model("model/main_5.h5")
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        if type(text) is float:
            text = ""
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m5.predict([seq, numerical_data], verbose=0, use_multiprocessing=True)
        print(seq.shape, numerical_data.shape, y, ans[0,0], "\n\n")
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[:500])

(1, 650) (1, 5) 6.607445106972985 4.56906 


(1, 650) (1, 5) 6.607445106972985 4.56906 


(1, 650) (1, 5) 6.998161370910088 5.149872 


(1, 650) (1, 5) 7.511502103994039 4.7837014 


(1, 650) (1, 5) 7.129246811243481 7.735744 


(1, 650) (1, 5) 8.7151200080925 5.9632263 


(1, 650) (1, 5) 8.7151200080925 5.2907996 


(1, 650) (1, 5) 6.3732589822241685 4.2612824 


(1, 650) (1, 5) 5.648499094454326 7.1917744 


(1, 650) (1, 5) 6.207302708748353 5.609754 


(1, 650) (1, 5) 5.025271726702564 5.4827995 


(1, 650) (1, 5) 7.106682637574135 5.135828 


(1, 650) (1, 5) 7.170997223562014 5.6656504 


(1, 650) (1, 5) 8.215482906027418 9.004035 


(1, 650) (1, 5) 6.71052568386338 9.863479 


(1, 650) (1, 5) 5.799266276454582 5.7667837 


(1, 650) (1, 5) 6.607445106972985 9.6097355 


(1, 650) (1, 5) 8.59389418591238 2.3477242 


(1, 650) (1, 5) 8.748627336347578 4.9842186 


(1, 650) (1, 5) 6.562115937095545 3.4804723 


(1, 650) (1, 5) 6.382068816107202 4.61909 


(1, 650) (1, 5) 8.709532579215

(1, 650) (1, 5) 8.022968937262847 7.694273 


(1, 650) (1, 5) 6.562115937095545 9.462663 


(1, 650) (1, 5) 8.735985510612334 10.256939 


(1, 650) (1, 5) 6.190498156775787 7.0517755 


(1, 650) (1, 5) 7.602357260334827 9.839076 


(1, 650) (1, 5) 8.74138683827009 8.557471 


(1, 650) (1, 5) 7.493869596318778 5.320137 


(1, 650) (1, 5) 6.759892234844327 6.397925 


(1, 650) (1, 5) 8.29845086437032 4.861979 


(1, 650) (1, 5) 5.95227783477598 2.3797846 


(1, 650) (1, 5) 8.482883456649951 6.183782 


(1, 650) (1, 5) 6.3732589822241685 3.6905873 


(1, 650) (1, 5) 7.648306726574672 4.2991686 


(1, 650) (1, 5) 7.493869596318778 5.320137 


(1, 650) (1, 5) 5.95227783477598 2.3797846 


(1, 650) (1, 5) 3.4718572059490853 4.98785 


(1, 650) (1, 5) 6.607445106972985 4.52077 


(1, 650) (1, 5) 7.921906947449914 5.128724 


(1, 650) (1, 5) 6.3995078738655895 3.3780093 


(1, 650) (1, 5) 7.92459471510565 4.6243715 


(1, 650) (1, 5) 1.9376946881169843 5.853043 


(1, 650) (1, 5) 5.83112765288

(1, 650) (1, 5) 5.717921805828719 4.977541 


(1, 650) (1, 5) 3.433064691451144 5.937667 


(1, 650) (1, 5) 6.181254371017418 4.961407 


(1, 650) (1, 5) 5.1385587970376365 7.7023916 


(1, 650) (1, 5) 6.765825685915322 5.405187 


(1, 650) (1, 5) 3.916649591116808 5.884642 


(1, 650) (1, 5) 4.878171957939991 4.33146 


(1, 650) (1, 5) 5.433997586477339 2.0619085 


(1, 650) (1, 5) 5.433997586477339 2.0619085 


(1, 650) (1, 5) 6.765825685915322 5.405187 


(1, 650) (1, 5) 8.323452146415569 8.797771 


(1, 650) (1, 5) 5.691791949528471 9.18215 


(1, 650) (1, 5) 5.47795598012679 8.728829 


(1, 650) (1, 5) 8.743005258264253 8.824189 


(1, 650) (1, 5) 5.675766931338512 7.680109 


(1, 650) (1, 5) 8.743005258264253 8.824189 


(1, 650) (1, 5) 8.154949057424771 7.735633 


(1, 650) (1, 5) 6.124057251494149 5.5029445 


(1, 650) (1, 5) 5.675766931338512 7.7234144 


(1, 650) (1, 5) 7.171574816814388 4.6899986 


(1, 650) (1, 5) 6.583882046776493 7.1387095 


(1, 650) (1, 5) 6.12747516941

In [97]:
m6 = keras.models.load_model("model/main_6.h5")
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        if type(text) is float:
            text = ""
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m6.predict([seq, numerical_data], verbose=0, use_multiprocessing=True)
        print(i, seq.shape, numerical_data.shape, y, ans[0,0])
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[:500])

(1, 650) (1, 5) 6.607445106972985 5.8127427 


(1, 650) (1, 5) 6.607445106972985 5.8127427 


(1, 650) (1, 5) 6.998161370910088 6.3570895 


(1, 650) (1, 5) 7.511502103994039 5.9039617 


(1, 650) (1, 5) 7.129246811243481 7.6674356 


(1, 650) (1, 5) 8.7151200080925 6.0513635 


(1, 650) (1, 5) 8.7151200080925 5.457527 


(1, 650) (1, 5) 6.3732589822241685 4.275187 


(1, 650) (1, 5) 5.648499094454326 6.918917 


(1, 650) (1, 5) 6.207302708748353 6.9319363 


(1, 650) (1, 5) 5.025271726702564 5.8217354 


(1, 650) (1, 5) 7.106682637574135 7.043701 


(1, 650) (1, 5) 7.170997223562014 6.8974485 


(1, 650) (1, 5) 8.215482906027418 8.468321 


(1, 650) (1, 5) 6.71052568386338 9.198353 


(1, 650) (1, 5) 5.799266276454582 5.665368 


(1, 650) (1, 5) 6.607445106972985 8.571775 


(1, 650) (1, 5) 8.59389418591238 4.373391 


(1, 650) (1, 5) 8.748627336347578 6.6808653 


(1, 650) (1, 5) 6.562115937095545 4.3464537 


(1, 650) (1, 5) 6.382068816107202 4.6809654 


(1, 650) (1, 5) 8.709532579

(1, 650) (1, 5) 8.022968937262847 7.6463 


(1, 650) (1, 5) 6.562115937095545 9.165465 


(1, 650) (1, 5) 8.735985510612334 9.488014 


(1, 650) (1, 5) 6.190498156775787 6.8543034 


(1, 650) (1, 5) 7.602357260334827 9.197855 


(1, 650) (1, 5) 8.74138683827009 8.150829 


(1, 650) (1, 5) 7.493869596318778 6.238433 


(1, 650) (1, 5) 6.759892234844327 7.24239 


(1, 650) (1, 5) 8.29845086437032 6.768985 


(1, 650) (1, 5) 5.95227783477598 2.3519144 


(1, 650) (1, 5) 8.482883456649951 7.0732946 


(1, 650) (1, 5) 6.3732589822241685 4.6448336 


(1, 650) (1, 5) 7.648306726574672 5.5866213 


(1, 650) (1, 5) 7.493869596318778 6.238433 


(1, 650) (1, 5) 5.95227783477598 2.3519144 


(1, 650) (1, 5) 3.4718572059490853 5.219888 


(1, 650) (1, 5) 6.607445106972985 5.22333 


(1, 650) (1, 5) 7.921906947449914 6.825315 


(1, 650) (1, 5) 6.3995078738655895 4.2979236 


(1, 650) (1, 5) 7.92459471510565 6.7460427 


(1, 650) (1, 5) 1.9376946881169843 6.620867 


(1, 650) (1, 5) 5.8311276528850

(1, 650) (1, 5) 5.717921805828719 6.6935964 


(1, 650) (1, 5) 3.433064691451144 6.781996 


(1, 650) (1, 5) 6.181254371017418 6.3474507 


(1, 650) (1, 5) 5.1385587970376365 7.6916113 


(1, 650) (1, 5) 6.765825685915322 6.5570393 


(1, 650) (1, 5) 3.916649591116808 6.342993 


(1, 650) (1, 5) 4.878171957939991 4.4645143 


(1, 650) (1, 5) 5.433997586477339 2.8469808 


(1, 650) (1, 5) 5.433997586477339 2.8469808 


(1, 650) (1, 5) 6.765825685915322 6.5570393 


(1, 650) (1, 5) 8.323452146415569 8.678534 


(1, 650) (1, 5) 5.691791949528471 7.9215364 


(1, 650) (1, 5) 5.47795598012679 8.49169 


(1, 650) (1, 5) 8.743005258264253 8.26296 


(1, 650) (1, 5) 5.675766931338512 7.8116164 


(1, 650) (1, 5) 8.743005258264253 8.26296 


(1, 650) (1, 5) 8.154949057424771 7.657869 


(1, 650) (1, 5) 6.124057251494149 5.581224 


(1, 650) (1, 5) 5.675766931338512 7.7559867 


(1, 650) (1, 5) 7.171574816814388 6.417901 


(1, 650) (1, 5) 6.583882046776493 6.8947163 


(1, 650) (1, 5) 6.1274751

In [98]:
m7 = keras.models.load_model("model/main_7.h5")
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        if type(text) is float:
            text = ""
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m7.predict([seq, numerical_data], verbose=0, use_multiprocessing=True)
        print(i, seq.shape, numerical_data.shape, y, ans[0,0])
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[:500])

0 (1, 650) (1, 5) 6.607445106972985 7.003138
1 (1, 650) (1, 5) 6.607445106972985 7.003138
2 (1, 650) (1, 5) 6.998161370910088 7.009486
3 (1, 650) (1, 5) 7.511502103994039 6.9518347
4 (1, 650) (1, 5) 7.129246811243481 7.6822543
5 (1, 650) (1, 5) 8.7151200080925 7.2385097
6 (1, 650) (1, 5) 8.7151200080925 7.186653
7 (1, 650) (1, 5) 6.3732589822241685 6.74552
8 (1, 650) (1, 5) 5.648499094454326 7.7585716
9 (1, 650) (1, 5) 6.207302708748353 7.025906
10 (1, 650) (1, 5) 5.025271726702564 7.018037
11 (1, 650) (1, 5) 7.106682637574135 8.773248
12 (1, 650) (1, 5) 7.170997223562014 7.075405
13 (1, 650) (1, 5) 8.215482906027418 8.212103
14 (1, 650) (1, 5) 6.71052568386338 8.219455
15 (1, 650) (1, 5) 5.799266276454582 7.3851404
16 (1, 650) (1, 5) 6.607445106972985 8.871988
17 (1, 650) (1, 5) 8.59389418591238 7.2948947
18 (1, 650) (1, 5) 8.748627336347578 6.6488757
19 (1, 650) (1, 5) 6.562115937095545 6.091563
20 (1, 650) (1, 5) 6.382068816107202 6.8755584
21 (1, 650) (1, 5) 8.709532579215772 8.176

176 (1, 650) (1, 5) 8.811472745889803 7.0709705
177 (1, 650) (1, 5) 8.022968937262847 7.655532
178 (1, 650) (1, 5) 6.562115937095545 8.94952
179 (1, 650) (1, 5) 8.735985510612334 8.678675
180 (1, 650) (1, 5) 6.190498156775787 7.801993
181 (1, 650) (1, 5) 7.602357260334827 8.83496
182 (1, 650) (1, 5) 8.74138683827009 8.619458
183 (1, 650) (1, 5) 7.493869596318778 7.0915895
184 (1, 650) (1, 5) 6.759892234844327 7.3903446
185 (1, 650) (1, 5) 8.29845086437032 7.061632
186 (1, 650) (1, 5) 5.95227783477598 5.222072
187 (1, 650) (1, 5) 8.482883456649951 7.3645787
188 (1, 650) (1, 5) 6.3732589822241685 6.2608137
189 (1, 650) (1, 5) 7.648306726574672 7.352452
190 (1, 650) (1, 5) 7.493869596318778 7.0915895
191 (1, 650) (1, 5) 5.95227783477598 5.222072
192 (1, 650) (1, 5) 3.4718572059490853 7.1732397
193 (1, 650) (1, 5) 6.607445106972985 6.5751038
194 (1, 650) (1, 5) 7.921906947449914 6.974619
195 (1, 650) (1, 5) 6.3995078738655895 5.869152
196 (1, 650) (1, 5) 7.92459471510565 6.7318544
197 (1, 

350 (1, 650) (1, 5) 8.853339021798474 6.322569
351 (1, 650) (1, 5) 7.625362103342714 9.0568075
352 (1, 650) (1, 5) 5.991446166933935 6.7322865
353 (1, 650) (1, 5) 6.245772451546388 8.84192
354 (1, 650) (1, 5) 5.717921805828719 6.9413395
355 (1, 650) (1, 5) 3.433064691451144 7.2713523
356 (1, 650) (1, 5) 6.181254371017418 7.1150746
357 (1, 650) (1, 5) 5.1385587970376365 7.650081
358 (1, 650) (1, 5) 6.765825685915322 7.27722
359 (1, 650) (1, 5) 3.916649591116808 6.32635
360 (1, 650) (1, 5) 4.878171957939991 6.781838
361 (1, 650) (1, 5) 5.433997586477339 6.0487475
362 (1, 650) (1, 5) 5.433997586477339 6.0487475
363 (1, 650) (1, 5) 6.765825685915322 7.27722
364 (1, 650) (1, 5) 8.323452146415569 8.099989
365 (1, 650) (1, 5) 5.691791949528471 9.0724325
366 (1, 650) (1, 5) 5.47795598012679 8.0972395
367 (1, 650) (1, 5) 8.743005258264253 8.749294
368 (1, 650) (1, 5) 5.675766931338512 7.7583766
369 (1, 650) (1, 5) 8.743005258264253 8.749294
370 (1, 650) (1, 5) 8.154949057424771 7.6770954
371 (1

In [99]:
m8 = keras.models.load_model("model/main_8.h5")
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        if type(text) is float:
            text = ""
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m8.predict([seq, numerical_data], verbose=0, use_multiprocessing=True)
        print(i, seq.shape, numerical_data.shape, y, ans[0,0])
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[:500])

0 (1, 650) (1, 5) 6.607445106972985 7.436115
1 (1, 650) (1, 5) 6.607445106972985 7.436115
2 (1, 650) (1, 5) 6.998161370910088 7.4556384
3 (1, 650) (1, 5) 7.511502103994039 7.398362
4 (1, 650) (1, 5) 7.129246811243481 7.6577344
5 (1, 650) (1, 5) 8.7151200080925 7.502333
6 (1, 650) (1, 5) 8.7151200080925 7.494796
7 (1, 650) (1, 5) 6.3732589822241685 7.363033
8 (1, 650) (1, 5) 5.648499094454326 7.556862
9 (1, 650) (1, 5) 6.207302708748353 7.524156
10 (1, 650) (1, 5) 5.025271726702564 7.4199495
11 (1, 650) (1, 5) 7.106682637574135 7.8191
12 (1, 650) (1, 5) 7.170997223562014 7.5480695
13 (1, 650) (1, 5) 8.215482906027418 7.8974915
14 (1, 650) (1, 5) 6.71052568386338 8.040422
15 (1, 650) (1, 5) 5.799266276454582 7.5417814
16 (1, 650) (1, 5) 6.607445106972985 8.047005
17 (1, 650) (1, 5) 8.59389418591238 7.3820224
18 (1, 650) (1, 5) 8.748627336347578 7.263766
19 (1, 650) (1, 5) 6.562115937095545 6.947698
20 (1, 650) (1, 5) 6.382068816107202 7.4309435
21 (1, 650) (1, 5) 8.709532579215772 7.9017

177 (1, 650) (1, 5) 8.022968937262847 7.6471696
178 (1, 650) (1, 5) 6.562115937095545 8.061745
179 (1, 650) (1, 5) 8.735985510612334 7.958565
180 (1, 650) (1, 5) 6.190498156775787 7.5711236
181 (1, 650) (1, 5) 7.602357260334827 8.042045
182 (1, 650) (1, 5) 8.74138683827009 7.918044
183 (1, 650) (1, 5) 7.493869596318778 7.4298444
184 (1, 650) (1, 5) 6.759892234844327 7.541405
185 (1, 650) (1, 5) 8.29845086437032 7.453797
186 (1, 650) (1, 5) 5.95227783477598 6.6451316
187 (1, 650) (1, 5) 8.482883456649951 7.5609207
188 (1, 650) (1, 5) 6.3732589822241685 7.164664
189 (1, 650) (1, 5) 7.648306726574672 7.3870206
190 (1, 650) (1, 5) 7.493869596318778 7.4298444
191 (1, 650) (1, 5) 5.95227783477598 6.6451316
192 (1, 650) (1, 5) 3.4718572059490853 7.5740705
193 (1, 650) (1, 5) 6.607445106972985 7.297549
194 (1, 650) (1, 5) 7.921906947449914 7.454442
195 (1, 650) (1, 5) 6.3995078738655895 6.716582
196 (1, 650) (1, 5) 7.92459471510565 7.2663302
197 (1, 650) (1, 5) 1.9376946881169843 7.5821285
198

350 (1, 650) (1, 5) 8.853339021798474 7.232566
351 (1, 650) (1, 5) 7.625362103342714 8.225359
352 (1, 650) (1, 5) 5.991446166933935 7.3521423
353 (1, 650) (1, 5) 6.245772451546388 8.038855
354 (1, 650) (1, 5) 5.717921805828719 7.4109383
355 (1, 650) (1, 5) 3.433064691451144 7.5616975
356 (1, 650) (1, 5) 6.181254371017418 7.4530697
357 (1, 650) (1, 5) 5.1385587970376365 7.6325927
358 (1, 650) (1, 5) 6.765825685915322 7.514774
359 (1, 650) (1, 5) 3.916649591116808 6.800159
360 (1, 650) (1, 5) 4.878171957939991 7.377387
361 (1, 650) (1, 5) 5.433997586477339 6.8552165
362 (1, 650) (1, 5) 5.433997586477339 6.8552165
363 (1, 650) (1, 5) 6.765825685915322 7.514774
364 (1, 650) (1, 5) 8.323452146415569 7.8061643
365 (1, 650) (1, 5) 5.691791949528471 8.105591
366 (1, 650) (1, 5) 5.47795598012679 7.8193955
367 (1, 650) (1, 5) 8.743005258264253 7.9704165
368 (1, 650) (1, 5) 5.675766931338512 7.6809506
369 (1, 650) (1, 5) 8.743005258264253 7.9704165
370 (1, 650) (1, 5) 8.154949057424771 7.654289
3

In [100]:
m9 = keras.models.load_model("model/main_9.h5")
def prepare_data(validate_data_sample):
    predicted_y = []
    true_y = []
    for i in range(validate_data_sample.shape[0]):
        data = validate_data_sample.iloc[i, :][["review_by_patient_filter", "effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]]
        text = data["review_by_patient_filter"]
        if type(text) is float:
            text = ""
        seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
        seq = pad_sequences([seq], maxlen=650)
        numerical_feature = list(data)
        numerical_data = np.array(numerical_feature[1:-1]).reshape(1, 5)
        y = numerical_feature[-1]
        ans = m9.predict([seq, numerical_data], verbose=0, use_multiprocessing=True)
        print(i, seq.shape, numerical_data.shape, y, ans[0,0])
        predicted_y.append(ans[0,0])
        true_y.append(y)
    print("mean sq", mean_squared_error(true_y, predicted_y))
    print("mean_absolute_error ",mean_absolute_error(true_y, predicted_y))
    print("r2 Score",r2_score(true_y, predicted_y))
prepare_data(validate_data[:500])

0 (1, 650) (1, 5) 6.607445106972985 7.466663
1 (1, 650) (1, 5) 6.607445106972985 7.466663
2 (1, 650) (1, 5) 6.998161370910088 7.4535747
3 (1, 650) (1, 5) 7.511502103994039 7.458475
4 (1, 650) (1, 5) 7.129246811243481 7.653077
5 (1, 650) (1, 5) 8.7151200080925 7.4945264
6 (1, 650) (1, 5) 8.7151200080925 7.4795012
7 (1, 650) (1, 5) 6.3732589822241685 7.4072695
8 (1, 650) (1, 5) 5.648499094454326 7.508538
9 (1, 650) (1, 5) 6.207302708748353 7.551034
10 (1, 650) (1, 5) 5.025271726702564 7.4527545
11 (1, 650) (1, 5) 7.106682637574135 7.8095174
12 (1, 650) (1, 5) 7.170997223562014 7.579618
13 (1, 650) (1, 5) 8.215482906027418 7.8857923
14 (1, 650) (1, 5) 6.71052568386338 8.03887
15 (1, 650) (1, 5) 5.799266276454582 7.527013
16 (1, 650) (1, 5) 6.607445106972985 7.9997983
17 (1, 650) (1, 5) 8.59389418591238 7.394622
18 (1, 650) (1, 5) 8.748627336347578 7.30909
19 (1, 650) (1, 5) 6.562115937095545 7.004216
20 (1, 650) (1, 5) 6.382068816107202 7.474176
21 (1, 650) (1, 5) 8.709532579215772 7.8904

176 (1, 650) (1, 5) 8.811472745889803 7.446461
177 (1, 650) (1, 5) 8.022968937262847 7.6444488
178 (1, 650) (1, 5) 6.562115937095545 8.015324
179 (1, 650) (1, 5) 8.735985510612334 7.92317
180 (1, 650) (1, 5) 6.190498156775787 7.5543585
181 (1, 650) (1, 5) 7.602357260334827 7.9996896
182 (1, 650) (1, 5) 8.74138683827009 7.881045
183 (1, 650) (1, 5) 7.493869596318778 7.4582543
184 (1, 650) (1, 5) 6.759892234844327 7.5290613
185 (1, 650) (1, 5) 8.29845086437032 7.4560523
186 (1, 650) (1, 5) 5.95227783477598 6.7820926
187 (1, 650) (1, 5) 8.482883456649951 7.5436287
188 (1, 650) (1, 5) 6.3732589822241685 7.238666
189 (1, 650) (1, 5) 7.648306726574672 7.428975
190 (1, 650) (1, 5) 7.493869596318778 7.4582543
191 (1, 650) (1, 5) 5.95227783477598 6.7820926
192 (1, 650) (1, 5) 3.4718572059490853 7.5894403
193 (1, 650) (1, 5) 6.607445106972985 7.3449903
194 (1, 650) (1, 5) 7.921906947449914 7.5137186
195 (1, 650) (1, 5) 6.3995078738655895 6.8096266
196 (1, 650) (1, 5) 7.92459471510565 7.315575
19

349 (1, 650) (1, 5) 8.233307958262657 6.6529036
350 (1, 650) (1, 5) 8.853339021798474 7.3274746
351 (1, 650) (1, 5) 7.625362103342714 8.181029
352 (1, 650) (1, 5) 5.991446166933935 7.4026675
353 (1, 650) (1, 5) 6.245772451546388 7.9952297
354 (1, 650) (1, 5) 5.717921805828719 7.4561753
355 (1, 650) (1, 5) 3.433064691451144 7.55114
356 (1, 650) (1, 5) 6.181254371017418 7.4475446
357 (1, 650) (1, 5) 5.1385587970376365 7.62971
358 (1, 650) (1, 5) 6.765825685915322 7.505036
359 (1, 650) (1, 5) 3.916649591116808 6.8676043
360 (1, 650) (1, 5) 4.878171957939991 7.427235
361 (1, 650) (1, 5) 5.433997586477339 6.9474716
362 (1, 650) (1, 5) 5.433997586477339 6.9474716
363 (1, 650) (1, 5) 6.765825685915322 7.505036
364 (1, 650) (1, 5) 8.323452146415569 7.7825375
365 (1, 650) (1, 5) 5.691791949528471 8.047192
366 (1, 650) (1, 5) 5.47795598012679 7.8122706
367 (1, 650) (1, 5) 8.743005258264253 7.9324546
368 (1, 650) (1, 5) 5.675766931338512 7.674867
369 (1, 650) (1, 5) 8.743005258264253 7.9324546
37